##**Implementation of CNN architecture for FashionMINST dataset**

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from keras import models, layers, applications
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, BatchNormalization, ReLU, Add

Using TensorFlow backend.


In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform lables to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Reshape the dataset into 4D array - 4th dimension number of datapoints
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

4423680/4422102 [==============================] - 0s 0us/step



**Below goes the simple CNN model code**


In [ ]:
basic_CNN_model = Sequential([
                    layers.Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"),
                    layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"),
                    layers.Dropout(0.3),
                    
                    layers.Conv2D(16, kernel_size=(4, 4), strides=(1, 1), activation='relu', padding="valid"),
                    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"),
                    
                    layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding="valid"),
                    layers.Dropout(0.2),
                    
                    layers.Flatten(),
                    layers.Dense(10, activation="softmax")
])

In [ ]:
basic_CNN_model.compile(loss=keras.losses.categorical_crossentropy, optimizer='ADAM', metrics=["accuracy"])
basic_CNN_model.summary()



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 6)         60        
_________________________________________________________________
average_pooling2d_2 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 16)        1552      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1, 1, 120)         48120     
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 1, 120)      

In [ ]:
hist_basic = basic_CNN_model.fit(x = x_train, y = y_train, epochs = 10, batch_size = 512, validation_data = (x_test, y_test), verbose = 1)
test_score = basic_CNN_model.evaluate(x_test, y_test)
print("\nTest loss of my basic CNN model {:.4f}, and accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.4302 - acc: 0.8400 - val_loss: 0.4128 - val_acc: 0.8493
Epoch 2/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.4218 - acc: 0.8430 - val_loss: 0.3946 - val_acc: 0.8574
Epoch 3/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.4156 - acc: 0.8467 - val_loss: 0.3971 - val_acc: 0.8555
Epoch 4/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.4104 - acc: 0.8481 - val_loss: 0.3925 - val_acc: 0.8560
Epoch 5/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.4026 - acc: 0.8508 - val_loss: 0.3833 - val_acc: 0.8591
Epoch 6/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.3974 - acc: 0.8537 - val_loss: 0.3824 - val_acc: 0.8604
Epoch 7/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.3931 - acc: 0.8541 - val_loss: 0.3657 - val_acc

**Below goes the code for complex CNN model where resnet and inception module is together**

In [ ]:
inputs = Input(x_train.shape[1:])
                    
# start of incepiton module

tower_one = layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(inputs)
tower_one = layers.Conv2D(6, (1,1), activation='relu', border_mode='same')(tower_one)
tower_two = layers.Conv2D(6, (1,1), activation='relu', border_mode='same')(inputs)
tower_two = layers.Conv2D(6, (3,3), activation='relu', border_mode='same')(tower_two)
tower_three = layers.Conv2D(6, (1,1), activation='relu', border_mode='same')(inputs)
tower_three = layers.Conv2D(6, (5,5), activation='relu', border_mode='same')(tower_three)
x = keras.layers.concatenate([tower_one, tower_two, tower_three], axis=3)

# end of inception module

# start of resnet module

h_1 = layers.Conv2D(6, (3,3), border_mode='same')(x)
h_1 = BatchNormalization()(h_1)
h_1 = ReLU()(h_1)
h_2 = layers.Conv2D(6, (3,3), border_mode='same')(h_1)
h_2 = BatchNormalization()(h_2)
shortcut = layers.Conv2D(6, (1,1), padding='same')(inputs)
shortcut = BatchNormalization()(shortcut)
output = Add()([h_2, shortcut])
x = layers.Flatten()(output)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

# end of the resnet module

complex_CNN_model = keras.Model(input = inputs, output = predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (1, 1), activation="relu", padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (1, 1), activation="relu", padding="same")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (3, 3), activation="relu", padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (1, 1), activation="relu", padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/lo

In [ ]:
complex_CNN_model.summary()
complex_CNN_model.compile(loss=keras.losses.categorical_crossentropy, optimizer='ADAM', metrics=["accuracy"])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 28, 28, 1)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 28, 28, 6)    12          input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 28, 28, 6)    12          input_2[0][0]                    
____________________________________________________________________________________________

In [ ]:
hist_complex = complex_CNN_model.fit(x = x_train, y = y_train, epochs = 10, batch_size = 512, validation_data = (x_test, y_test), verbose = 1)
test_score = complex_CNN_model.evaluate(x_test, y_test)
print("\nTest loss of my complex CNN model {:.4f}, and accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0710 - acc: 0.9747 - val_loss: 0.4679 - val_acc: 0.8721
Epoch 2/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0565 - acc: 0.9809 - val_loss: 0.3944 - val_acc: 0.8925
Epoch 3/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0467 - acc: 0.9844 - val_loss: 0.4521 - val_acc: 0.8881
Epoch 4/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0405 - acc: 0.9864 - val_loss: 0.5452 - val_acc: 0.8717
Epoch 5/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0363 - acc: 0.9879 - val_loss: 0.4828 - val_acc: 0.8950
Epoch 6/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0260 - acc: 0.9922 - val_loss: 0.4832 - val_acc: 0.8941
Epoch 7/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0248 - acc: 0.9923 - val_loss: 0.5432 - val_acc